# Air Quality Monitor

## Introduzione

*Air Quality Monitor è un progetto universitario che nasce con lo scopo di tenere traccia della qualità dell’aria in ogni momento delle 
maggiori città del mondo.*

Il progetto è reso possibile grazie all’utilizzo di <u>API</u> di [IQAir](https://www.iqair.com/it/) che vengono fornite tramite registrazione gratuita al servizio, quindi con l’utilizzo di una chiave che viene fornita successivamente e che ha la validità di un anno.

Fra le varie tecnologie utilizzate vi sono **Docker**, **Logstash**, **Kafka** $\rightarrow$ **TODO**

Il progetto è visualizzabile al seguente [link](https://github.com/Picred/air-quality-monitor) di Github dove vi è la guida per l’avvio nel file `README.md`

L’intero progetto è sostenuto da Docker. Tramite questa tecnologia è possibile gestire i cosiddetti *container* che permettono di immagazzinare il tutto in luogo facilmente riproducibile su qualsiasi altra macchina che sfrutti Docker.

## Il servizio di IQAir

![IQAir Logo](https://upload.wikimedia.org/wikipedia/en/thumb/5/5f/IQAir_logo.svg/360px-IQAir_logo.svg.png)

IQAir è un servizio specializzo nell’analisi della qualità dell’aria. In particolare si occupa di:
1.	offrire strumenti e servizi per **monitorare l’inquinamento atmosferico** in tempo reale. Questo include l’*Indice di Qualità dell’Aria (AQI)*, che fornisce informazioni sulla qualità dell’aria in diverse località;
2.	produrre purificatori d’aria avanzati per migliorare la qualità dell’aria negli ambienti interni. Questi dispositivi rimuovono particelle sospese nell’aria, come **polveri sottili** (*PM2.5*), allergeni e altre sostanze nocive;
3.	classificare le città inquinate pubblicando tali classifiche basate sui livelli di PM2.5. Questo sensibilizza le persone sull’importanza di affrontare l’inquinamento atmosferico e di adottare misure per ridurlo.

Per l’utilizzo delle API è presenta la documentazione online che è consultabile al [seguente link](https://api-docs.iqair.com/#f6cadcfb-1522-40a3-8d38-1b98a92faf0c). In breve, è possibile eseguire alcune operazioni (di tipo *GET*) che permettono di ottenere dati riguardanti:
-	Tutti gli stati supportati;
-	Tutte le regioni supportate in uno stato;
-	Tutte le città supportate in una regione;
-	La città più vicina geograficamente (*tramite analisi dell’indirizzo IP*);
-	La città più vicina geograficamente (*tramite le coordinate GPS*);
-	Una città specificata fra le disponibili.

## Ingestion Manager (python)

E' stato scritto un file `ingestion-manager.py` in modo tale da gestire più da vicino quello che accade. 

In tale script viene usata la libreria **requests** per fare delle richieste HTTP ai vari URL che permettono di usare le API di IQAir per recuperare i dati di nostro interesse.

Per quanto riguarda la connessione fra i dati ricevuti dalle chiamate API e Logstash si è optato, come specificato prima, di utilizzare la libreria [pylogbeat](https://pypi.org/project/pylogbeat/) in modo da semplificare la complessità del codice. All’interno dell’`ingestion-manager.py` vi sono anche dei controlli che riguardano le variabili d’ambiente.

### Funzionamento e logica

Il file `ingestion-manager.py` viene usato per fare le chiamate GET al server IQAir. In particolar modo è necessaria la registrazione al servizio per l’ottenimento dell’**API_KEY** usata per comporre i link corretti. Di base, nel container Docker che viene creato, vengono impostate delle variabili d’ambiente di default, tra cui:
* `DATA_ACTION`, che permette di definire l’azione da eseguire sui dati che è una di quelle che è fornita dalle API (descritte nell’Introduzione) che possono essere:
  - `ALL_COUNTRIES` per ottenere tutte le nazioni supportate;
  - `NEAREST_IP_CITY` per ottenere la città più vicina (tramite IP);
  - `NEAREST_GPS_CITY` per ottenere la città più vicina (tramite GPS);
  - `ALL_STATES_BY_COUNTRY` per ottenere tuStte le regioni in una nazione;
  - `ALL_CITIES_BY_STATE_COUNTRY` per ottenere tutte le città in una specifica regione e nazione.
* `API_KEY` banalmente quella fornita in seguito alla registrazione su IQAir
* `COUNTRY_NAME` nome della nazione da analizzare
* `STATE_NAME` nome della regione da analizzare
* `GPS_LAT` latitudine della posizione da analizzare
* `GPS_LON` longitudine della posizione da analizzare
* `CITY_TO_SCAN` città specifica da analizzare
> *N.B: Alla scelta del nome della nazione-regione-città da analizzare bisogna sceglierne una fra le disponibili a tale analisi.*

In seguito alla scelta della DATA_ACTION da eseguire è necessario verificare che le dipendenze fra le variabili d'ambiente sia rispettata. Esse sono ben definite come nella seguente tabella:


|           DATA_ACTION        |               DIPENDENZE          |
| -----------------------------| ----------------------------------|
|          ALL_COUNTRIES       |               API_KEY             |
|         NEAREST_IP_CITY      |               API_KEY             |
|       ALL_STATES_BY_COUNTRY  |           API_KEY, STATE_NAME     |
| ALL_CITIES_BY_STATE_COUNTRY  |  API_KEY, STATE_NAME, COUNTRY_NAME|
|          NEAREST_GPS_CITY    |      API_KEY, GPS_LAT, GPS_LON    |

#### Esempi
1.	Per vedere **tutte le nazioni supportate** serve impostare `DATA_ACTION="ALL_COUNTRIES"` e `API_KEY="{YOUR_KEY}"`
2.	Per vedere una **determinata posizione geografica** serve impostare `DATA_ACTION="NEAREST_GPS_CITY"`, `API_KEY="{key}"`, `GPS_LAT="{latitudine}"` e `GPS_LON="{longitudine}"`

> *All’interno dello script vi sono anche delle verifiche che guidano l’utente finale al giusto setup dell’applicazione, come per esempio la verifica dell’avvenuta impostazione della `API_KEY` o che sia stato correttamente avviato il container Logstash dal quale l’ingestion-manager dipende.*

## Logstash

![Logstash Logo](https://cdn.icon-icons.com/icons2/2699/PNG/256/elasticco_logstash_logo_icon_170185.png)

La scelta di tale tecnologia è basata sul fatto che il file di configurazione è **molto semplice da scrivere**, in quanto si limita ad essere un file come quello sottostante:

In [ ]:
input{
    ...
}

filter{
    ...
}

output{
    ...
}

Nel nostro caso l’input avverrà tramite [Beats](https://www.elastic.co/beats) che è un plugin di input di Logstash che, rispetto a `TCP`:
- permette di semplificare il processo di configurazione che avviene manualmente con `TCP`;
- gestisce **automaticamente** questa struttura dati, **semplificando l'elaborazione e l'analisi dei dati.**

Inoltre, è stata fatta questa scelta perché con `TCP` vi era un errore del tipo:
<code><font color="red">Caused by: org.logstash.beats.InvalidFrameProtocolException: Invalid version of beats protocol: 123</font></code>

Logstash ha i <span style="font-weight:bold; color:green;">file di configurazione molto intuitivi e brevi</span> rispetto alle altre tecnologie di Data Ingestion ma, a differenza loro, crea degli <span style="color:red; font-weight:bold;">eventi molto più complessi</span> con delle ridondanze. A discapito di questo si preferisce l’uso di una configurazione più semplice

Per queste ragioni vi è stato un adattamento anche da parte dello script Python ingestion-manager.py che **inizialmente** utilizzava i metodi forniti dalla libreria `socket` di Python come indicato sotto:

In [ ]:
import socket
sockfd = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sockfd.connect(('logstash', 5044))

Per quanto riguarda il file di configurazione in `air-quality-monitor/logstash/pipeline/from_python_to_kafka.conf` sono stati impostai i seguenti *plugin*:
- **input**: `beats` che è in ascolto sulla porta specificata e attende dati da client esterni (*`ingestion_manager.py` nel nostro caso*);
- **output**: è direzionato su un Topic di Kafka, in particolare al container che esegue il Server di Kafka. In particolare viene specificato anche il Topic che viene automaticamente creato se esso non esiste.